In [1]:
from dotenv import load_dotenv
import os, json
import requests

load_dotenv()
api_key = os.getenv("API_KEY", None)
project_id = os.getenv("PROJECT_ID", None)
ibm_cloud_iam_url = os.getenv("IAM_IBM_CLOUD_URL", None)

creds = {
    "url"    : "https://us-south.ml.cloud.ibm.com",
    "apikey" : api_key
}

params = {
    "decoding_method":"greedy",
    "max_new_tokens":3000,
    "min_new_tokens":1,
    # "temperature":0.1,
    "top_k":50,
    "top_p":1,
     "repetition_penalty": 1.1,
    "stop_sequences":["```","<|user|>","<|assistant|>"],
}

In [2]:
# Prepare the payload and headers
payload = {
    "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
    "apikey": api_key
}
headers = {
    'Content-Type': "application/x-www-form-urlencoded"
}

# Make a POST request while ignoring SSL certificate verification
try:
    response = requests.post(f"https://{ibm_cloud_iam_url}/identity/token", data=payload, headers=headers, verify=False)
    
    # Check if the request was successful
    response.raise_for_status()

    # Parse the JSON response
    decoded_json = response.json()
    access_token = decoded_json["access_token"]
    # print(f"Access Token: {access_token}")
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'iam.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [3]:
import requests

GRANITE_3B_CODE_INSTRUCT = "ibm/granite-3b-code-instruct"
GRANITE_8B_CODE_INSTRUCT = "ibm/granite-8b-code-instruct"
GRANITE_20B_CODE_INSTRUCT = "ibm/granite-20b-code-instruct"
GRANITE_34B_CODE_INSTRUCT = "ibm/granite-34b-code-instruct"
GRANITE_3_8B_INSTRUCT = "ibm/granite-3-8b-instruct"

def watsonx_code_gen(prompt,model_id):

    url = "https://us-south.ml.cloud.ibm.com/ml/v1/text/generation_stream?version=2023-05-29"

    body = {
        "input": prompt,
        "parameters": params,
        "model_id": model_id,
        "project_id": project_id
    }

    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.post(
        url,
        headers=headers,
        json=body,
        stream=True
    )

    if response.status_code != 200:
        raise Exception("Non-200 response: " + str(response.text))

    answer = ""
    # Stream the response
    for line in response.iter_lines():
        if line:  # Ensure the line is not empty
            decoded_line = line.decode("utf-8").strip()
            
            # Check if the line starts with "data: "
            if decoded_line.startswith("data: "):
                json_data = decoded_line[len("data: "):]  # Remove the "data: " prefix
                
                try:
                    # Attempt to load the JSON data
                    data = json.loads(json_data)
                    generated_text = data.get("results", "")
                    answer += generated_text[0]["generated_text"]
                    # print(generated_text[0]["generated_text"],end="")  # Print or process the generated text as needed
                except json.JSONDecodeError:
                    print("Failed to decode JSON:", json_data)
                except Exception as e:
                    print("An error occurred:", e)

    return answer


In [4]:
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
import os
from dotenv import load_dotenv


load_dotenv()
api_key = os.getenv("API_KEY", None)
project_id = os.getenv("PROJECT_ID", None)

creds = {
    "url"    : "https://us-south.ml.cloud.ibm.com",
    "apikey" : api_key
}

params = {
    GenParams.DECODING_METHOD:"greedy",
    GenParams.MAX_NEW_TOKENS:4000,
    GenParams.MIN_NEW_TOKENS:1,
    # "temperature":0.1,
    GenParams.TOP_K:50,
    GenParams.TOP_P:1,
    GenParams.STOP_SEQUENCES:["```","[/INST]","<|user|>","<|endoftext|>","<|assistant|>"],
}

model = ModelInference(model_id="ibm/granite-3-8b-instruct", credentials=creds, params=params, project_id=project_id)

In [5]:
model_id = GRANITE_3_8B_INSTRUCT
language = "typescript"
documentframework = "TypeDoc"
testframework = "Jest"

In [6]:
prompt = f"""<|user|>
Please generate a sample {language} code:
-give more detail and complex logic.
-generate the code only.
-dont generate explanation.
<|assistant|>
```{language}
"""

sample_code = model.generate_text(prompt).replace("```","")
print(sample_code)

import { Injectable } from '@angular/core';
import { HttpClient } from '@angular/common/http';
import { Observable } from 'rxjs';
import { catchError, map } from 'rxjs/operators';

interface User {
  id: number;
  name: string;
  email: string;
}

@Injectable({
  providedIn: 'root'
})
export class UserService {
  private apiUrl = 'https://api.example.com/users';

  constructor(private http: HttpClient) { }

  getUser(id: number): Observable<User> {
    return this.http.get<User>(`${this.apiUrl}/${id}`).pipe(
      map(user => ({ ...user, isLoaded: true })),
      catchError(this.handleError('getUser', []))
    );
  }

  private handleError<T>(operation = 'operation', result?: T) {
    return (error: any): Observable<T> => {
      console.error(error);
      return of(result as T);
    };
  }

  getUsers(): Observable<User[]> {
    return this.http.get<User[]>(this.apiUrl).pipe(
      map(users => users.map(user => ({ ...user, isLoaded: true }))),
      catchError(this.handleError('getU

In [13]:
sample_code = """import { Injectable } from '@angular/core';
import { HttpClient } from '@angular/common/http';
import { Observable } from 'rxjs';
import { catchError, map } from 'rxjs/operators';

interface User {
  id: number;
  name: string;
  email: string;
}

@Injectable({
  providedIn: 'root'
})
export class UserService {
  private apiUrl = 'https://api.example.com/users';

  constructor(private http: HttpClient) { }

  getUser(id: number): Observable<User> {
    return this.http.get<User>(`${this.apiUrl}/${id}`).pipe(
      map(user => ({ ...user, isLoaded: true })),
      catchError(this.handleError('getUser', []))
    );
  }

  private handleError<T>(operation = 'operation', result?: T) {
    return (error: any): Observable<T> => {
      console.error(error);
      return of(result as T);
    };
  }

  getUsers(): Observable<User[]> {
    return this.http.get<User[]>(this.apiUrl).pipe(
      map(users => users.map(user => ({ ...user, isLoaded: true }))),
      catchError(this.handleError('getUsers', []))
    );
  }

  createUser(user: User): Observable<User> {
    return this.http.post<User>(this.apiUrl, user).pipe(
      map(createdUser => ({ ...createdUser, isLoaded: true })),
      catchError(this.handleError('createUser', {}))
    );
  }

  updateUser(user: User): Observable<User> {
    const updatedUser = { ...user, isLoaded: true };
    return this.http.put<User>(`${this.apiUrl}/${user.id}`, updatedUser).pipe(
      map(updatedUser => ({ ...updatedUser, isLoaded: true })),
      catchError(this.handleError('updateUser', {}))
    );
  }

  deleteUser(id: number): Observable<void> {
    return this.http.delete<void>(`${this.apiUrl}/${id}`).pipe(
      catchError(this.handleError('deleteUser', void 0))
    );
  }
}
"""

bad_code = """import { Injectable } from '@angular/core';
import { HttpClient } from '@angular/common/http';
import { Observable } from 'rxjs';
import { catchError, map } from 'rxjs/operators';

interface User {
  id: number;
  name: string;
  email: string;
}

@Injectable({
  providedIn: 'root'
})
export class UserService {
  private apiUrl = 'https://api.example.com/users';

  constructor(private http: HttpClient) { }

  getUser(id: number): Observable<User> {
    return this.http.get<ded: true })),
      catchError(this.handleError('getUser', []))
    );
  }

  private handleError<T>(operation = 'operation', result?: T) {
    return (error: any): Observable<T> => {
      console.error(error);
      return of(result as T);
    };
  }r[]>(this.apiUrl).pipe(
      map(users => users.map(user => ({ ...user, isLoaded: true }))),
      catchError(this.handleError('getUsers', []))
    );
  }

  createUser(user: User): Observable<User> {
    return this.http.post<User>(this.apiUrl, user).pipe(
      map(createdUser => ({ ...createdUser, isLoaded: true })),
      catchError(this.handleError('createUser', {}))
    );
  }

  updateUser(user: User): Observable<User> {
    const updatedUser = { ...user, isLoaded: true };
    return this.http.put<Us
  }

  deleteUser(id: number): Observable<void> {
    return this.http.delete<void>(`${this.apiUrl}/${id}`).pipe(
      catchError(this.handleError('deleteUser', void 0))
    );
  }
}
"""

gnerate explanation

In [7]:
prompt = f"""<|user|>
Please provide a detailed explanation for the following {language} code:
-ensure have new line after every full stop.
-dont generate code.

###
code: `{sample_code}`
<|assistant|>
explanation:
"""

answer = model.generate_text(prompt)
print(answer)


This TypeScript code defines an Angular service called `UserService` that handles CRUD (Create, Read, Update, Delete) operations for a `User` object. Here's a breakdown of the code:

1. **Imports**: The service imports necessary modules and interfaces from Angular and RxJS.

2. **User Interface**: An interface `User` is defined with properties `id`, `name`, and `email`.

3. **UserService**: The `UserService` is decorated with `@Injectable` to make it injectable as a service. It's provided at the root level, meaning it can be injected anywhere in the application.

4. **Constructor**: The constructor injects the `HttpClient` service, which is used to make HTTP requests.

5. **Methods**:

   - `getUser(id: number)`: This method retrieves a single user by their ID. It returns an observable that emits the user object with an additional `isLoaded` property set to `true`.

   - `getUsers()`: This method retrieves a list of all users. It returns an observable that emits an array of user objec

generate document

In [9]:
prompt = f"""<|user|>
generate {documentframework} document for the {language} provided.
-generate the {language} code only.
{language}: `{sample_code}`
-generate the document.
<|assistant|>
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


/**
 * UserService provides methods for interacting with the user API.
 */
import { Injectable } from '@angular/core';
import { HttpClient } from '@angular/common/http';
import { Observable } from 'rxjs';
import { catchError, map } from 'rxjs/operators';

interface User {
  id: number;
  name: string;
  email: string;
}

@Injectable({
  providedIn: 'root'
})
export class UserService {
  private apiUrl = 'https://api.example.com/users';

  constructor(private http: HttpClient) { }

  /**
   * Gets a user by ID.
   * @param id The ID of the user to retrieve.
   * @returns An observable that emits the user data.
   */
  getUser(id: number): Observable<User> {
    return this.http.get<User>(`${this.apiUrl}/${id}`).pipe(
      map(user => ({ ...user, isLoaded: true })),
      catchError(this.handleError('getUser', []))
    );
  }

  /**
   * Gets a list of users.
   * @returns An observable that emits an array of user data.
   */
  getUsers(): Observable<User[]> {
    return this.http.get<

score, review and improve

In [10]:
prompt = f"""<|user|>
you are {language} expert. you need to review the following {language} code. 
-please understand the {language} code provided in detail.
-please review the code in completeness, logic, performance and quality.
-make it more object oriented.
-give a score out of 10 for the quality of code.
-please provide constructive suggestion in detail.
-generate the review and suggestion only.
{language} code: `{sample_code}`
-DONT generate the improve code directly.
-dont generate code.
<|assistant|>
score and review:"""

suggestion = model.generate_text(prompt)
print(suggestion)



The provided TypeScript code is well-structured and follows best practices for Angular services. It uses Angular's HttpClient for making HTTP requests and employs RxJS operators for handling responses. The code is also object-oriented, with methods for getting, creating, updating, and deleting users.

However, there are a few areas for improvement:

1. Error handling: While the `handleError` method is used to handle errors, it might be better to throw a more specific error type instead of returning an empty observable. This would make it easier to handle errors in the components that use this service.

2. Type safety: The `createUser` and `updateUser` methods return an object with the `isLoaded` property set to `true`. However, this property is not defined in the `User` interface. It would be better to add this property to the interface or remove it from the returned object.

3. Code duplication: The `map` operator is used to add the `isLoaded` property to the response in several met

In [11]:
prompt = f"""<|user|>
improve the {language} provided base on the review provided.
-understand the review suggestion in detail.
-adopt the review suggestion if applicable.
-dont generate explanation.
###
original {language} code: `{sample_code}`
review suggestion: `{suggestion}`
<|assistant|>
improved {language}
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


import { Injectable } from '@angular/core';
import { HttpClient } from '@angular/common/http';
import { Observable } from 'rxjs';
import { catchError, map, throwError } from 'rxjs/operators';

interface User {
  id: number;
  name: string;
  email: string;
}

@Injectable({
  providedIn: 'root'
})
export class UserService {
  private apiUrl = 'https://api.example.com/users';

  constructor(private http: HttpClient) { }

  getUserById(id: number): Observable<User> {
    return this.http.get<User>(`${this.apiUrl}/${id}`).pipe(
      map(user => ({ ...user, isLoaded: true })),
      catchError(this.handleError<User>('getUserById', []))
    );
  }

  getUsersList(): Observable<User[]> {
    return this.http.get<User[]>(this.apiUrl).pipe(
      map(users => users.map(user => ({ ...user, isLoaded: true }))),
      catchError(this.handleError<User[]>('getUsersList', []))
    );
  }

  createUser(user: User): Observable<User> {
    return this.http.post<User>(this.apiUrl, user).pipe(
      map

fix bad code

In [14]:
prompt = f"""<|user|>
fix the {language} provided.
-understand the review suggestion in detail.
-identify if any problem.
-fix the problem if any.
-add explanation as inline comment.

###
{language}: `{bad_code}`
<|assistant|>
fixed version with inline comment at the point it fix.
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


import { Injectable } from '@angular/core';
import { HttpClient } from '@angular/common/http';
import { Observable } from 'rxjs';
import { catchError, map } from 'rxjs/operators';

interface User {
  id: number;
  name: string;
  email: string;
}

@Injectable({
  providedIn: 'root'
})
export class UserService {
  private apiUrl = 'https://api.example.com/users';

  constructor(private http: HttpClient) { }

  getUser(id: number): Observable<User> {
    return this.http.get<User>(`${this.apiUrl}/${id}`).pipe(
      catchError(this.handleError('getUser', []))
    );
  }

  private handleError<T>(operation = 'operation', result?: T) {
    return (error: any): Observable<T> => {
      console.error(error);
      return of(result as T);
    };
  }

  getUsers(): Observable<User[]> {
    return this.http.get<User[]>(this.apiUrl).pipe(
      map(users => users.map(user => ({ ...user, isLoaded: true }))),
      catchError(this.handleError('getUsers', []))
    );
  }

  createUser(user: User):

generate diagram

In [15]:
prompt = f"""<|user|>
generate diagram for the {language} code provided.
-understand the review suggestion in detail.
-use simple name for node.
-generate in mermaid.
-dont repeating.
-dont generation explanation.
{language}: `{sample_code}`
<|assistant|>
```mermaid"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


graph TD
A[UserService] --> B[getUser(id: number): Observable<User>]
A --> C[getUsers(): Observable<User[]>]
A --> D[createUser(user: User): Observable<User>]
A --> E[updateUser(user: User): Observable<User>]
A --> F[deleteUser(id: number): Observable<void>]
B --> G[http.get<User>(`${this.apiUrl}/${id}`)]
C --> H[http.get<User[]>(this.apiUrl)]
D --> I[http.post<User>(this.apiUrl, user)]
E --> J[http.put<User>(`${this.apiUrl}/${user.id}`, updatedUser)]
F --> K[http.delete<void>(`${this.apiUrl}/${id}`)]
G --> L[map(user => ({ ...user, isLoaded: true }))]
H --> M[map(users => users.map(user => ({ ...user, isLoaded: true })))]
I --> N[map(createdUser => ({ ...createdUser, isLoaded: true }))]
J --> O[map(updatedUser => ({ ...updatedUser, isLoaded: true }))]
K --> P[catchError(this.handleError('deleteUser', void 0))]
L --> Q[catchError(this.handleError('getUser', []))]
M --> R[catchError(this.handleError('getUsers', []))]
N --> S[catchError(this.handleError('createUser', {}))]
O --> T[catch

conversion

In [18]:
target_language = "golang"

prompt = f"""<|user|>
rewrite the {language} code provided to {target_language}.
-understand the code provided in detail.
-understand how to map technology and syntax to the target {target_language} langauge.
-dont generation explanation.
{language}: `{sample_code}`
<|assistant|>
```{target_language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


package main

import (
	"net/http"

	"github.com/tidwall/gjson"
)

type User struct {
	ID    int    `json:"id"`
	Name  string `json:"name"`
	Email string `json:"email"`
}

type UserService struct {
	apiUrl string
}

func NewUserService(apiUrl string) *UserService {
	return &UserService{apiUrl: apiUrl}
}

func (us *UserService) GetUser(id int) (*User, error) {
	resp, err := http.Get(us.apiUrl + "/" + strconv.Itoa(id))
	if err != nil {
		return nil, err
	}
	defer resp.Body.Close()

	var user User
	err = gjson.Get(resp.Body, "#.id,#.name,#.email").Scan(&user)
	if err != nil {
		return nil, err
	}

	user.IsLoaded = true
	return &user, nil
}

func (us *UserService) GetUsers() ([]*User, error) {
	resp, err := http.Get(us.apiUrl)
	if err != nil {
		return nil, err
	}
	defer resp.Body.Close()

	var users []User
	err = gjson.Get(resp.Body, "#").Array().Each(&users, func(i int, v gjson.Result) bool {
		user := User{}
		err = v.Get("id").Int(&user.ID)
		if err != nil {
			return false
		}
		err 

generate api definition

In [19]:
prompt = f"""<|user|>
generate a API definition in swagger for the {language} code provided.
-dont generate explanation.
{language}: `{sample_code}`
<|assistant|>
```yaml
"""

answer = model.generate_text(prompt).replace("```","")
print(answer)

openapi: 3.0.0
info:
  title: UserService API
  version: 1.0.0
servers:
  - url: https://api.example.com/users
paths:
  /{id}:
    get:
      summary: Get a user by ID
      parameters:
        - name: id
          in: path
          required: true
          schema:
            type: integer
            format: int64
      responses:
        '200':
          description: A user object
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/User'
    post:
      summary: Create a new user
      requestBody:
        required: true
        content:
          application/json:
            schema:
              $ref: '#/components/schemas/User'
      responses:
        '201':
          description: The created user object
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/User'
  /:
    get:
      summary: Get a list of users
      responses:
        '200':
          de

generate test case

In [20]:
prompt = f"""<|user|>
generate Test Cases in {testframework} for the {language} code provided.
-focus on boundary cases.
-focus on exception handling
-generate the test only.
{language}: `{sample_code}`
<|assistant|>
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


import { TestBed } from '@angular/core/testing';
import { HttpClientTestingModule, HttpTestingController } from '@angular/common/http/testing';
import { UserService } from './user.service';
import { User } from './user.model';

describe('UserService', () => {
  let service: UserService;
  let httpMock: HttpTestingController;

  beforeEach(() => {
    TestBed.configureTestingModule({
      imports: [HttpClientTestingModule],
      providers: [UserService]
    });
    service = TestBed.inject(UserService);
    httpMock = TestBed.inject(HttpTestingController);
  });

  afterEach(() => {
    httpMock.verify();
  });

  it('should get user by id', () => {
    const mockUser: User = { id: 1, name: 'John Doe', email: 'john.doe@example.com' };
    service.getUser(1).subscribe(user => expect(user).toEqual(mockUser));

    const req = httpMock.expectOne(`${service.apiUrl}/1`);
    expect(req.request.method).toBe('GET');
    req.flush(mockUser);
  });

  it('should get users', () => {
    const 